In [9]:
# import libraries
from patchify import patchify, unpatchify
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage import img_as_ubyte, exposure, io
import cv2
import os
from datetime import datetime

In [11]:
%%time

image = 'images/S1A_IW_GRDH_1SDV_20150419T155633_20150419T155658_005555_0071AF_7CF1_Orb_NR_Cal_EC.tiff'
full_img = io.imread(image)

# print(full_img.shape, full_img.dtype, type(full_img))
h, w = full_img.shape[0], full_img.shape[1]

threshold = 0.04
full_img[full_img > threshold] = threshold

# # # display image
# # plt.figure(figsize=(10,10))
# # plt.imshow(full_img, cmap='gray') # display the image
# # plt.axis('off')
# # plt.show()


# patches with overlap patch_size > step_size

# # patch and step sizes
patch_size = 4096
step_size= (2048, 2048)

# padding image before creating patches in order to cover entire image

# Pad the image with zeros
img_padded = np.pad(full_img, ((0, patch_size - full_img.shape[0] % patch_size),
                           (0, patch_size - full_img.shape[1] % patch_size)), mode='constant')


# Extract patches using patchify
patches_padded = patchify(img_padded, (patch_size, patch_size), step=step_size)
patches_padded.shape

# # displaying patches
# fig, ax = plt.subplots(patches_padded.shape[0], patches_padded.shape[1], figsize=(8,8), facecolor='gray')
# for i in range(patches_padded.shape[0]):
#     for j in range(patches_padded.shape[1]):
#         ax[i][j].imshow(patches_padded[i][j], cmap='gray')
#         ax[i][j].axis('off')
#         plt.subplots_adjust(wspace=0.1, hspace=0.1)
#         plt.subplots_adjust(wspace=0.1, hspace=0.1)

# Save patches to folder

# Set the folder path
folder_path = 'images/padded_overlap2/'

# Check if the folder exists, create it if not
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Set the threshold value
thresh = 50

for i in range(patches_padded.shape[0]):
    for j in range(patches_padded.shape[1]):
        single_patch_img = patches_padded[i, j, :, :]
        
        # Apply thresholding to enhance contrast
        single_patch_img[single_patch_img > thresh] = thresh
        
        # Normalize the image
        single_patch_img = cv2.normalize(single_patch_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
        
        # Save the image
        if not cv2.imwrite(folder_path + 'image_' + '_' + str(i) + str(j) + '.jpg', single_patch_img):
            raise Exception("Could not write the image")         

CPU times: user 2min 2s, sys: 26.4 s, total: 2min 29s
Wall time: 2min 46s
